In [ ]:
# import pdb; pdb.set_trace()
# import pudb; pudb.set_trace()

# df.style.background_gradient(
#     cmap='coolwarm', 
#     axis=None, 
#     vmin=-1, 
#     vmax=1
# )  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'

# pd.set_option('precision', 2)

# .plot_bokeh(sizing_mode="scale_height")
# .plot_bokeh(kind='hist', bins=1000, sizing_mode="scale_height", xlim=[0, .1])

# Imports

In [15]:
%load_ext autoreload
%autoreload 2

import datetime as dt
import pickle
import numpy as np
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 99)
import pandas_bokeh
pandas_bokeh.output_file("/tmp/bokeh_output.html")
import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]
import ray

from data_layer import arrow_dataset, storage_adaptor
from tick_filter import streaming_tick_filter
from tick_sampler import streaming_tick_sampler, daily_stats
from workflows import sampler_task, sampler_flow
from utilities import date_fu, globals_unsafe as g
from data_layer import storage_adaptor, fsspec_factory, data_access

config = {
    'meta': {
        'symbol': 'VTI',
        'start_date': '2020-01-01',
        'end_date': '2020-01-20',
        'config_id': 'renko_v1',
    },
    'filter': {
        'mad_value_winlen': 22,
        'mad_deviation_winlen': 1111,
        'mad_k': 17,
        'jma_winlen': 7,
        'jma_power': 2,
    },
    'sampler': {
        'renko_return': 'price_jma_return',
        'renko_size': 0.1,  # for simple runs
        'renko_reveral_multiple': 2,
        'renko_range_frac': 22,
        'renko_range_min_pct_value': 0.03,  # % of symbol value enforced as min renko size
        'max_duration_td': dt.timedelta(minutes=33),
        'min_duration_td': dt.timedelta(seconds=33),
        'min_tick_count': 33,
        'add_label': True,
        'reward_ratios': list(np.arange(2, 11, 0.5)),
    }
}

prefix_1 = f"/tick_samples/{config['meta']['config_id']}/bar_date"

prefix_2 = f"/tick_samples/{config['meta']['config_id']}/bars_df"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_access.list_symbol_dates('VTI', prefix_1, source='remote')

['2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07', '2020-01-08']

In [ ]:
data_access.list_symbol_storage('VTI', prefix_2, source='both')

# Sample tick dates

In [3]:
ray.init(dashboard_port=1111, ignore_reinit_error=True)

2021-05-29 22:23:16,508	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:1111


{'node_ip_address': '192.168.1.66',
 'raylet_ip_address': '192.168.1.66',
 'redis_address': '192.168.1.66:6379',
 'object_store_address': '/tmp/ray/session_2021-05-29_22-23-14_056739_67401/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-29_22-23-14_056739_67401/sockets/raylet',
 'webui_url': '127.0.0.1:1111',
 'session_dir': '/tmp/ray/session_2021-05-29_22-23-14_056739_67401',
 'metrics_export_port': 52671,
 'node_id': '754b0e6f67b2b6e20f87111a47ceef93773f88d15877c7f48861c832'}

In [ ]:
ray.shutdown()

In [16]:
daily_stats_df = sampler_flow.get_dates_from_config(config)

In [ ]:
bar_dates = sampler_flow.run(config, ray_on=True)

2 dates scheduled
(pid=67421) running VTI 2020-01-09
(pid=67422) running VTI 2020-01-10


In [ ]:
n = 1

# bar_dates[n]['ticks_df'][['price', 'price_jma']].plot_bokeh(sizing_mode="scale_height")
bar_dates[n]['ticks_df'][['price', 'price_jma']].plot()

In [ ]:
bar_dates[n]['ticks_df'][['price_jma']].plot()

In [ ]:
bar_dates[n]['ticks_df'].status.value_counts() / len(bar_dates[n]['ticks_df'])

In [ ]:
bar_dates[n]['bars_df'][['price_low', 'price_high', 'price_close']].plot()

In [ ]:
# fill daily gaps
stacked_df = stacked.fill_gaps_dates(bds, fill_col='price_vwap')

stacked_df[['price_high','price_low','price_close', 'price_vwap']].plot_bokeh(sizing_mode="scale_height")

# stacked stats
stats_df = stacked.stacked_df_stats(stacked_df)

stats_df.describe()